In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB1, EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6, EfficientNetB7
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

# Define EfficientNet models and corresponding input sizes
efficientnet_versions = {
    'EfficientNetB1': (EfficientNetB1, 240)   
}

# Path to dataset
base_path = '/kaggle/input/diffmango224x224/AMBADIFUSEDATASET224X224'
train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'val')
test_path = os.path.join(base_path, 'test')

# Store results
results = []

for model_name, (model_class, img_size) in efficientnet_versions.items():
    print(f"\nTraining {model_name} with input size {img_size}x{img_size}")

    # Data generators (no traditional augmentation)
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_gen = datagen.flow_from_directory(
        train_path,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical'
    )
    val_gen = datagen.flow_from_directory(
        val_path,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical'
    )
    test_gen = datagen.flow_from_directory(
        test_path,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    base_model = model_class(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    output = Dense(train_gen.num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10,  # You can increase epochs
        verbose=1
    )

    # Evaluation
    test_loss, test_acc = model.evaluate(test_gen)
    print(f"Test Accuracy for {model_name}: {test_acc:.4f}")

    # Predict and Classification Report
    y_pred = model.predict(test_gen)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = test_gen.classes
    class_labels = list(test_gen.class_indices.keys())

    report = classification_report(y_true, y_pred_classes, target_names=class_labels, output_dict=True)
    cm = confusion_matrix(y_true, y_pred_classes)

    results.append({
        'Model': model_name,
        'Input Size': img_size,
        'Test Accuracy': test_acc,
        'Classification Report': report,
        'Confusion Matrix': cm
    })

# Convert accuracy to DataFrame and display
df_results = pd.DataFrame([
    {
        'Model': r['Model'],
        'Input Size': r['Input Size'],
        'Test Accuracy': r['Test Accuracy']
    } for r in results
])
print("\nSummary of Results:")
print(df_results)

# Optional: Save the report
# df_results.to_csv("efficientnet_results.csv", index=False)


2025-07-14 14:09:44.780343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752502184.972460      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752502185.031670      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



Training EfficientNetB1 with input size 240x240
Found 46646 images belonging to 6 classes.
Found 9993 images belonging to 6 classes.
Found 10001 images belonging to 6 classes.


I0000 00:00:1752502246.024514      35 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1752502272.550374     114 service.cc:148] XLA service 0x7cc640001ee0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752502272.551246     114 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752502275.151640     114 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/1458 ━━━━━━━━━━━━━━━━━━━━ 15:28:52 38s/step - accuracy: 0.1875 - loss: 2.0317

I0000 00:00:1752502288.266643     114 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1458/1458 ━━━━━━━━━━━━━━━━━━━━ 507s 322ms/step - accuracy: 0.8047 - loss: 0.5623 - val_accuracy: 0.9517 - val_loss: 0.1586
Epoch 2/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 172s 118ms/step - accuracy: 0.9263 - loss: 0.2190 - val_accuracy: 0.9711 - val_loss: 0.1106
Epoch 3/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 165s 113ms/step - accuracy: 0.9352 - loss: 0.1939 - val_accuracy: 0.9797 - val_loss: 0.0913
Epoch 4/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 167s 114ms/step - accuracy: 0.9402 - loss: 0.1765 - val_accuracy: 0.9770 - val_loss: 0.0838
Epoch 5/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 171s 117ms/step - accuracy: 0.9365 - loss: 0.1808 - val_accuracy: 0.9809 - val_loss: 0.0768
Epoch 6/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 172s 118ms/step - accuracy: 0.9423 - loss: 0.1704 - val_accuracy: 0.9782 - val_loss: 0.0768
Epoch 7/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 168s 115ms/step - accuracy: 0.9433 - loss: 0.1645 - val_accuracy: 0.9827 - val_loss: 0.0701
Epoch 8/10
1458/1458 ━━━━━━━━━━━━━━━━━━━━ 169s 116ms/step - accuracy: 0.9